# Download crystal structures from PDB

Download PDB structures of a target protein, saving them as `pdb` files.

In [1]:
import pandas as pd
import numpy as np
import nglview as nv
from pathlib import Path
import sys
sys.path.insert(0, '../..')
from helper_modules.get_pdb_ids_from_uniport import *

## Inputs
This notebook requires to specify the following values
- `prot_name`: the name of the protein.
- `uniprot_id`: Uniprot Accession number.
- `ref_struc_id`: PDB id of the reference structure.
- `ref_ligand_name`: Three letters name of the cocrystalized ligand in the reference structure.


In [2]:
prot_name       = 'egfr'
uniprot_id      = 'P00533'
ref_struc_id    = '7a2a'
ref_ligand_name = '7G9'

In [3]:
seq_prot = get_seq_from_uniprot(uniprot_id)
print(seq_prot)
print(f'\nThere are {len(seq_prot)} residues.')

MRPSGTAGAALLALLAALCPASRALEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYVQRNYDLSFLKTIQEVAGYVLIALNTVERIPLENLQIIRGNMYYENSYALAVLSNYDANKTGLKELPMRNLQEILHGAVRFSNNPALCNVESIQWRDIVSSDFLSNMSMDFQNHLGSCQKCDPSCPNGSCWGAGEENCQKLTKIICAQQCSGRCRGKSPSDCCHNQCAAGCTGPRESDCLVCRKFRDEATCKDTCPPLMLYNPTTYQMDVNPEGKYSFGATCVKKCPRNYVVTDHGSCVRACGADSYEMEEDGVRKCKKCEGPCRKVCNGIGIGEFKDSLSINATNIKHFKNCTSISGDLHILPVAFRGDSFTHTPPLDPQELDILKTVKEITGFLLIQAWPENRTDLHAFENLEIIRGRTKQHGQFSLAVVSLNITSLGLRSLKEISDGDVIISGNKNLCYANTINWKKLFGTSGQKTKIISNRGENSCKATGQVCHALCSPEGCWGPEPRDCVSCRNVSRGRECVDKCNLLEGEPREFVENSECIQCHPECLPQAMNITCTGRGPDNCIQCAHYIDGPHCVKTCPAGVMGENNTLVWKYADAGHVCHLCHPNCTYGCTGPGLEGCPTNGPKIPSIATGMVGALLLLLVVALGIGLFMRRRHIVRKRTLRRLLQERELVEPLTPSGEAPNQALLRILKETEFKKIKVLGSGAFGTVYKGLWIPEGEKVKIPVAIKELREATSPKANKEILDEAYVMASVDNPHVCRLLGICLTSTVQLITQLMPFGCLLDYVREHKDNIGSQYLLNWCVQIAKGMNYLEDRRLVHRDLAARNVLVKTPQHVKITDFGLAKLLGAEEKEYHAEGGKVPIKWMALESILHRIYTHQSDVWSYGVTVWELMTFGSKPYDGIPASEISSILEKGERLPQPPICTIDVYMIMVKCWMIDADSRPKFRELIIEFSKMARDPQRYLVIQGDERMHLPSPTDSNFYRA

- Get from the Uniprot database all the associated PDB structures to the given Uniprot id.

In [4]:
# Get the pdbids and some metadata from uniprot
df_pdb_ids = pdb_ids_from_uniprot(uniprot_id)
df_pdb_ids.head()

,pdb_id,method,resolution,chain,start,end,seq_len
0,1ivo,X-ray,3.30,A,25,646,622
1,1m14,X-ray,2.60,A,695,1022,328
2,1m17,X-ray,2.60,A,695,1022,328
3,1mox,X-ray,2.50,A,25,525,501
4,1nql,X-ray,2.80,A,25,642,618


In [5]:
df_pdb_ids.shape

(221, 7)

## Get structural information from the reference structure

### For EGFR only

<mark>NOTE:</mark> As show in the `df_pdb_ids` table, the structures vary in their sequence coverage. The following cells aim to preserve only those structures containing the binding site of the protein by using the reference structure, which PDB id was provided above.

In [15]:
# Get the reference structure and its sequence and residue positions
seq_cry, positions_cry = get_structure_sequence(ref_struc_id)
print(len(seq_cry))
print(positions_cry[0], positions_cry[-1])
seq_cry

266
700 984


'NQALLRILKETEFKKIKVLGSGAFGTVYKGLWIPEGEKVKIPVAIKELREATSANKEILDEAYVMASVDNPHVCRLLGICLTSTVQLIMQLMPFGCLLDYVREHKDNIGSQYLLNWCVQIAKGMNYLEDRRLVHRDLAARNVLVKTPQHVKITDFGLVPIKWMALESILHRIYTHQSDVWSYGVTVWELMTFGSKPYDGIPASEISSILEKGERLPQPPICTIDVYMIMRKCWMIDADSRPKFRELIIEFSKMARDPQRYLVIQGD'

In [19]:
# Get the pdb id of those structures matching the positions of the 
# reference structure
df_sel_pdbids = get_useful_pdbids(df_pdb_ids, 
                                  positions_cry,
                                  thr_tol = (3,3))
df_sel_pdbids

,pdb_id,method,resolution,chain,start,end,seq_len
1,1m14,X-ray,2.60,A,695,1022,328
2,1m17,X-ray,2.60,A,695,1022,328
5,1xkk,X-ray,2.40,A,695,1022,328
8,2eb2,X-ray,2.50,A,695,1022,328
9,2eb3,X-ray,2.84,A,695,1022,328
...,...,...,...,...,...,...,...
216,6vhn,X-ray,2.40,A,696,1022,327
217,6vhp,X-ray,3.60,A,696,1022,327
218,6z4b,X-ray,2.50,A,695,1022,328
219,6z4d,X-ray,2.00,A,695,1022,328


In [8]:
ref_prot = parsePDB(ref_struc_id).select('chain A')
pocket_sel = ref_prot.select(
    f'within 7 of resname {ref_ligand_name} and protein')
pocket_residues = [str(i) 
                   for i in np.unique(
                       pocket_sel.getResnums())
                  ]
pocket_residues_str = ' '.join(pocket_residues)
lig_resnum = ref_prot.select(
                       f'resname {ref_ligand_name}'
                 ).getResnums()[0]

- Visualize using `nglview`

In [9]:
view = nv.show_prody(ref_prot)
view.clear_representations()
view.add_representation('cartoon', 
    selection='protein and chain A', color='orange')
view.add_cartoon(
    selection = pocket_residues_str, 
    color='cyan')
view.add_licorice(
    selection = pocket_residues_str, 
    color='cyan')
view.add_representation(
    'spacefill', selection = f'{lig_resnum}')
view.parameters = {
    "fogNear": 0, "fogFar": 100,
    "backgroundColor": "black",
}
view

NGLWidget()

In [10]:
view.render_image()

Image(value=b'', width='99%')

In [11]:
view._display_image()

## Download the PDB structures
1. Create the output directories

In [17]:
# Define the output directories
OUT_MAIN       = './pdb_structures'
OUT_DIR_RAW    = f'./{OUT_MAIN}/raw_structures'
OUT_DIR_CHAINS = f'./{OUT_MAIN}/pdb_chains'
OUT_DIR_LIGS   = f'./{OUT_MAIN}/raw_ligands'

for path in [OUT_DIR_RAW, OUT_DIR_CHAINS, OUT_DIR_LIGS]:
    Path(path).mkdir(parents = True, exist_ok = True)

2. Use `prody` to download and process the pdb chain belonging to the target protein.

In [18]:
from prody import parsePDB, matchAlign, writePDB

# We will use the reference structure to define the start and end
ref_chain = ref_prot.select('protein and chain A')
# HARDCODED
start_res = 685
end_res = 950

# Overlap and Sequence Identity thresholds
overlap = 80
seqid   = 95
residues_algn = pocket_residues_str 

# Iterate over the list of pdb_ids and download the structures
for i, (pdb_id, chid) in df_sel_pdbids[['pdb_id', 'chain']].iterrows():
    # Download the full structure
    raw_pdb = parsePDB(pdb_id, folder = OUT_DIR_RAW)
    # Keep the protein chain
    pdb_chain = raw_pdb.select(f'protein and chain {chid}')
    pdb_chain.setChids("A")
    # If the structure has more 
    min_start_res = pdb_chain.getResnums()[0]
    _start_res    = min(min_start_res, start_res)
    # Keep the same starting and ending points as the reference
    pdb_chain = pdb_chain.select(f'resnum {_start_res}to{end_res}')
    # Make the structure alignment regarding the ref_prot
    pdb_alg = matchAlign(
                mobile = pdb_chain,
                target = ref_chain,
                # If the overlap or the seqid thresholds are not met
                # the alignment will return None
                overlap = overlap,
                tarsel  = f'chain A ca and resnum {residues_algn}'
              )
    # If the alignment retuned None, skip the next steps
    if not pdb_alg:
        continue
    # Get the aligned structure
    protein = pdb_alg[0]
    # Set the same chain id (A) for all structures
    protein.setChids("A")
    # Save the PDB chain
    chain_pdb_file = f'{OUT_DIR_CHAINS}/{pdb_id}_A.pdb'
    writePDB(chain_pdb_file, protein)
    # Save the non proteic atoms (raw ligands)
    ligand = raw_pdb.select('hetatm')
    if ligand:
        raw_ligand_file = f'{OUT_DIR_LIGS}/{pdb_id}_lig.pdb'
        writePDB(raw_ligand_file, ligand)

Finished!